# Machine Learning
- Este jupyter notebook será utilizado para crear un modelo de recomendación consumible por la API


# Funcion Recomendacion item-item

In [3]:
#Importamos las librerias que vamos a usar
import pandas as pd
import numpy as np
from numpy import random
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
# Creamos Dummies para despues crear la matriz similitud del coseno 
games_ML = pd.read_parquet('src/games_ML.parquet')
games_ML.head(3)

,title,id,genre
0,Lost Summoner Kitty,761140,Action
0,Lost Summoner Kitty,761140,Casual
0,Lost Summoner Kitty,761140,Indie


In [5]:
#Creación dummies
games_ML= pd.get_dummies(games_ML, columns=["genre"], prefix="")

games_ML= games_ML.groupby(["id","title"]).sum().reset_index()

games_ML.head(4)

,id,title,"_""1990s""","_""Beat em up""","_""Shoot Em Up""",_2D,_3D Platformer,_Accounting,_Action,_Action RPG,...,_Top-Down Shooter,_Tower Defense,_Twin Stick Shooter,_Utilities,_Video Production,_Violent,_Visual Novel,_War,_Web Publishing,_Zombies
0,10,Counter-Strike,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
similitudes = pd.DataFrame(cosine_similarity(games_ML.iloc[:,3:]))
#similitudes.to_parquet('src/similitudes.parquet')

In [11]:
def recommend_games(id: int):
    # Verifica si el juego con game_id existe en df_games
    game = games_ML[games_ML['id'] == id]

    if game.empty:
        return("El juego '{id}' no posee registros.")
    
    # Obtiene el índice del juego dado
    idx = game.index[0]

    # Toma una muestra aleatoria del DataFrame df_games
    sample_size = 2000  # Define el tamaño de la muestra (ajusta según sea necesario)
    df_sample = games_ML.sample(n=sample_size, random_state=42)  # Ajusta la semilla aleatoria según sea necesario

    # Calcula la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([games_ML.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtiene las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordena los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtiene los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Lista de juegos similares (solo nombres)
    similar_game_names = df_sample['title'].iloc[similar_game_indices].tolist()

    return {f"Juegos Similares": similar_game_names}

In [12]:
recommend_games(20)

{'Juegos Similares': ['Streets of Rage 3',
  'Battlezone - Classic Neon (Skin)',
  'The Stalin Subway: Red Veil',
  'Strawberry Banana Surprise (Y)',
  'Marvel vs. Capcom: Infinite - Dante Nephilim Costume']}